## Training UNet With CelebA Dataset
### By 10 epochs, 3200steps.

In [1]:
import os
import sys
import datetime

import PIL.Image as Image
import matplotlib.pyplot as plt
%matplotlib inline

ROOT_DIR = os.path.join(os.getcwd(), '../..')
sys.path.append(ROOT_DIR)

In [2]:
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler


# from keras_segmentation.models.unet import unet as M

from keras_segmentation.models._unet import unet as M
from keras_segmentation.data_utils.data_loader import image_segmentation_generator, \
    image_segmentation_generator2



Using TensorFlow backend.
/home/jho/anaconda3/envs/tf10-py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jho/anaconda3/envs/tf10-py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jho/anaconda3/envs/tf10-py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jho/anaconda3/

In [3]:
now = datetime.datetime.now()
NOW = "{:%Y%m%dT%H%M}".format(now)

OUT_DIR = os.path.join(ROOT_DIR, 'out')
CHECKPOINTS_DIR = os.path.join(OUT_DIR, 'checkpoints')
PREDICTIONS_DIR = os.path.join(OUT_DIR, 'predictions')
LOGS_DIR = os.path.join(OUT_DIR, 'logs')
PLOTS_DIR = os.path.join(OUT_DIR, 'plots')

DATASET_DIR = os.path.join(ROOT_DIR, 'dataset/celeba')

TRAIN_IMAGES = os.path.join(DATASET_DIR, 'train/original')
TRAIN_ANNOTATIONS = os.path.join(DATASET_DIR, 'train/mask_')
VAL_IMAGES = os.path.join(DATASET_DIR, 'val/original')
VAL_ANNOTATIONS = os.path.join(DATASET_DIR, 'val/mask_')

In [4]:
## Configurations
N_CLASSES = 3

# I_HEIGHT = 416
# I_WIDTH = 608

I_HEIGHT = 218
I_WIDTH = 178

STEPS_PER_EPOCH = 320
EPOCHS = 10
BATCH_SIZE = 2

MODEL_NAME = 'unet'

In [5]:
tag = 'celeba/{name}/ep{epochs}/st{steps}'.format(
    name=MODEL_NAME,
    epochs=EPOCHS,
    steps=STEPS_PER_EPOCH
)

PLOT_PATH = os.path.join(PLOTS_DIR, 'model_{}.svg'.format(MODEL_NAME))
CHECKPOINTS_DIR = os.path.join(CHECKPOINTS_DIR, tag)
PREDICTIONS_DIR = os.path.join(PREDICTIONS_DIR, tag)
LOGS_DIR = os.path.join(LOGS_DIR, tag)
if not os.path.exists(CHECKPOINTS_DIR):
    os.makedirs(CHECKPOINTS_DIR)
if not os.path.exists(PREDICTIONS_DIR):
    os.makedirs(PREDICTIONS_DIR)
if not os.path.exists(LOGS_DIR):
    os.makedirs(LOGS_DIR)
    
dir_configuration = """
NOW = {now}
tag = {tag}
out_dir : {out_dir}
checkpoints_dir : {checkpoints_dir}
logs_dir : {logs_dir}
predictions_dir : {predictions_dir}

dataset_dir : {dataset_dir}
train_images : {train_images}
train_annotations : {train_annotations}
val_images : {val_images}
val_annotations : {val_annotations}
""".format(
    now=NOW,
    tag=tag,
    out_dir=OUT_DIR,
    checkpoints_dir=CHECKPOINTS_DIR,
    logs_dir=LOGS_DIR,
    predictions_dir=PREDICTIONS_DIR,

    dataset_dir=DATASET_DIR,
    train_images=TRAIN_IMAGES,
    train_annotations=TRAIN_ANNOTATIONS,
    val_images=VAL_IMAGES,
    val_annotations=VAL_ANNOTATIONS,
)
print(dir_configuration)


NOW = 20200420T1743
tag = celeba/unet/ep10/st320
out_dir : /home/jho/projects/image-segmentation-keras/train_faces/unet/../../out
checkpoints_dir : /home/jho/projects/image-segmentation-keras/train_faces/unet/../../out/checkpoints/celeba/unet/ep10/st320
logs_dir : /home/jho/projects/image-segmentation-keras/train_faces/unet/../../out/logs/celeba/unet/ep10/st320
predictions_dir : /home/jho/projects/image-segmentation-keras/train_faces/unet/../../out/predictions/celeba/unet/ep10/st320

dataset_dir : /home/jho/projects/image-segmentation-keras/train_faces/unet/../../dataset/celeba
train_images : /home/jho/projects/image-segmentation-keras/train_faces/unet/../../dataset/celeba/train/original
train_annotations : /home/jho/projects/image-segmentation-keras/train_faces/unet/../../dataset/celeba/train/mask_
val_images : /home/jho/projects/image-segmentation-keras/train_faces/unet/../../dataset/celeba/val/original
val_annotations : /home/jho/projects/image-segmentation-keras/train_faces/unet/.

In [6]:
train_gen = image_segmentation_generator(
    TRAIN_IMAGES, TRAIN_ANNOTATIONS, BATCH_SIZE, N_CLASSES,
    I_HEIGHT, I_WIDTH, I_HEIGHT, I_WIDTH)
val_gen = image_segmentation_generator(
    VAL_IMAGES, VAL_ANNOTATIONS, BATCH_SIZE, N_CLASSES,
    I_HEIGHT, I_WIDTH, I_HEIGHT, I_WIDTH)

In [7]:
# Define callbacks
tensorboard_callback = TensorBoard(
            log_dir=LOGS_DIR,
            histogram_freq=0,
            write_images=True
) if LOGS_DIR != '' else None

checkpoints_path = os.path.join(CHECKPOINTS_DIR, '{}.h5'.format(NOW))
checkpoint_callback = ModelCheckpoint(
            checkpoints_path, 
            monitor='val_loss', 
            verbose=1,
            save_best_only=True)

callbacks = [
    tensorboard_callback,
    checkpoint_callback,
]  

In [8]:
# Define model
# model = M(n_classes=N_CLASSES, input_height=I_HEIGHT, input_width=I_WIDTH)
model = M(input_size=(I_HEIGHT, I_WIDTH, 3))
model.summary()



from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model


plot_model(model, to_file=PLOT_PATH)
# SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))


Instructions for updating:
keep_dims is deprecated, use keepdims instead
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 218, 178, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 256, 256, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 256, 256, 64)  1792        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 256, 256, 64)  36928       conv2d_1[0][0]                   
__________________

/home/jho/projects/image-segmentation-keras/train_faces/unet/../../keras_segmentation/models/_unet.py:76: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("cr...)`
  model = Model(input = inputs, output = reshape2)


In [9]:
# Train
model.fit_generator(
    train_gen,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    callbacks=callbacks,
    validation_data=val_gen,
    validation_steps=200)

Epoch 1/10


ValueError: Error when checking target: expected cropping2d_1 to have 4 dimensions, but got array with shape (2, 38804, 3)

In [ ]:
import random
val_list = os.listdir(VAL_IMAGES)
rand_val = random.choice(val_list)[:-4]
print(rand_val)

sample_test_image = os.path.join(VAL_IMAGES, rand_val + '.jpg')
sample_test_annotaion = os.path.join(VAL_ANNOTATIONS, rand_val + '.bmp')
out_fname = os.path.join(PREDICTIONS_DIR, 'out_{}_{}.png'.format(rand_val, NOW))

In [ ]:
# Out
out = model.predict_generator(
    val_gen,
    steps=355
    verbose=1
)

In [ ]:
plt.imshow(Image.open(sample_test_image))

In [ ]:
plt.imshow(Image.open(sample_test_annotaion))

In [ ]:
plt.imshow(out)

In [ ]:
# evaluating the model
evaluation = model.evaluate_segmentation(inp_images_dir=VAL_IMAGES, annotations_dir=VAL_ANNOTATIONS)
print(evaluation)
